## Tree-like Device Topology Reconstruction Using Link Prediction Methods 

I worked on this research projects of a group of 3.

The code below **ONLY** covers the part that I have worked on.
- **This code only includes node2vec based model**


- **Aim**: Find missing edges between routers and base station throught the microwave
- **Dataset Used**: Huawei's tree-like mobile broadband topology dataset
- **Model used**: 
    - **node2vec with logistic regression**
    - **node2vec with Light GBM**
    - SEAL
    - HGCN 

In [2]:
import scipy
import mat4py as mp
from itertools import chain
import math
import random
import os.path as osp
import pandas as pd
import numpy as np
import networkx as nx
import torch
from torch_geometric import utils
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch_geometric.data import InMemoryDataset
from sklearn.metrics import roc_auc_score
from scipy.sparse.csgraph import shortest_path
import torch
import torch.nn.functional as F
from torch.nn import BCEWithLogitsLoss
from torch.nn import ModuleList, Linear, Conv1d, MaxPool1d
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, global_sort_pool
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch_geometric.utils import (negative_sampling, add_self_loops,
                                   train_test_split_edges, k_hop_subgraph,
                                   to_scipy_sparse_matrix)
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

import os
import networkx as nx
import numpy as np
import pandas as pd

from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph
from stellargraph import datasets
from IPython.display import display, HTML


df_net = pd.read_csv("With_Super_Node_Remove_Duplicat.csv")
df_nodes = pd.read_csv("Nodes.csv")
G = nx.from_pandas_edgelist(df_net,'Source','Dest')
node_list = list(df_nodes.index)
adj_G = nx.to_numpy_matrix(G, nodelist = node_list)

The Mobile broadband topology dataset from Huawei is about telecom device network. 

There are four types of devices in the network, which are **nodes**:
1. Super-router
2. Router
3. Microwave 
4. Nodeb

The information flow among devices is super-router to router, router to microwave and microwave to nodeb. There is one super-router in the network connecting all the routers, and one router will connect to multiple microwaves, and microwave can connect to microwaves as well as nodeb.

**Edges are information flows among devices**. 

There are 5346 nodes in the network including one super-router. However, we found some nodes with two attributes, so we decided to consider them as noisy data and deleted these nodes with its related flows. After removing all the noisy data, there are 4619 nodes and 4677 edges in the network.

In [3]:
adj_G.shape

(4619, 4619)

In [4]:

import pandas as pd
import numpy as np
import random
import networkx as nx
from tqdm import tqdm
import re
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

### Negative Samples

In [8]:
all_unconnected_pairs = []

# traverse adjacency matrix
offset = 0
for i in tqdm(range(4619)):
  for j in range(0,4619):
    if i != j:
      if nx.shortest_path_length(G, i, j) <=2:
        if adj_G[i,j] == 0:
          all_unconnected_pairs.append([node_list[i],node_list[j]])

  offset = offset + 1

100%|██████████| 4619/4619 [50:49<00:00,  1.51it/s]


In [9]:
len(all_unconnected_pairs)

15456

In [10]:
#Unconnected link -> make into dataframe 
node_1_unlinked = [i[0] for i in all_unconnected_pairs] #source node
node_2_unlinked = [i[1] for i in all_unconnected_pairs] #dest node

data = pd.DataFrame({'node_1':node_1_unlinked, 
                     'node_2':node_2_unlinked})

# add target variable 'link'
data['link'] = 0

### Positive Samples

we will randomly drop some of the edges from the graph. However, randomly removing edges may result in cutting off loosely connected nodes and fragments of the graph. This is something that we have to take care of. We have to make sure that in the process of dropping edges, all the nodes of the graph should remain connected.

In the code block below, we will first check if dropping a node pair results in the splitting of the graph (number_connected_components > 1) or reduction in the number of nodes. If both things do not happen, then we drop that node pair and repeat the same process with the next node pair.

In [11]:
hw_df = pd.DataFrame(pd.concat([df_net['Source'], df_net['Dest']],axis=1))
hw_df.rename(columns = {'Source':'node_1', 'Dest':'node_2'}, inplace=True)
hw_df.head()

,node_1,node_2
0,1002,0
1,2333,0
2,1436,0
3,2181,0
4,2279,0


In [12]:
initial_node_count = len(G.nodes)

hw_df_temp = hw_df.copy()

# empty list to store removable links
omissible_links_index = []

for i in tqdm(range(17983)):
  
  # remove a node pair and build a new graph
  G_temp = nx.from_pandas_edgelist(hw_df_temp.drop(index = i), "node_1", "node_2", create_using=nx.Graph())
  
  # check there is no spliting of graph and number of nodes is same
  if (nx.number_connected_components(G_temp) == 1) and (len(G_temp.nodes) == initial_node_count):
    omissible_links_index.append(i)
    hw_df_temp = hw_df_temp.drop(index = i)

100%|██████████| 17983/17983 [12:16<00:00, 24.42it/s]


In [13]:
len(omissible_links_index)

13365

In [14]:
# create dataframe of removable edges
hw_df_ghost = hw_df.loc[omissible_links_index]

# add the target variable 'link'
hw_df_ghost['link'] = 1

data = data.append(hw_df_ghost[['node_1', 'node_2', 'link']], ignore_index=True)

In [15]:
data['link'].value_counts()

0    15456
1    13365
Name: link, dtype: int64

## Feature Extraction ; node2vec


node2vec is a framework for learning continuous feature representations for nodes in networks. It maps nodes into low dimension using random walks that maximizes likelihood optimization problem.

One of the important parameters for node2vec is p and q that guides the walk. p is the return parameter and q is the in-out parameter.

In [16]:
# drop removable edges
hw_df_partial = hw_df.drop(index=hw_df_ghost.index.values)

# build graph
G_data = nx.from_pandas_edgelist(hw_df_partial, "node_1", "node_2", create_using=nx.Graph())

In [17]:
from node2vec import Node2Vec

# Generate walks
node2vec = Node2Vec(G_data, dimensions=100, walk_length=16, num_walks=50)

# train node2vec model
n2w_model = node2vec.fit(window=7, min_count=1)

Generating walks (CPU: 1): 100%|██████████| 50/50 [02:08<00:00,  2.62s/it]


we will apply the trained node2vec model on each and every node pair in the dataframe ‘data’. To compute the features of a pair or an edge, we will add up the features of the nodes in that pair:

In [18]:
x = [(n2w_model[str(i)]+n2w_model[str(j)]) for i,j in zip(data['node_1'], data['node_2'])]

/Users/kanghyojeong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


## Link Prediction Model

In [19]:
xtrain, xtest, ytrain, ytest = train_test_split(np.array(x), data['link'], 
                                                test_size = 0.3, 
                                                random_state = 35)

### 1. logistic regression

In [20]:

lr = LogisticRegression(class_weight="balanced")

lr.fit(xtrain, ytrain)

/Users/kanghyojeong/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [21]:
predictions = lr.predict_proba(xtest)
predictions_train = lr.predict_proba(xtrain)

In [22]:
roc_auc_score(ytrain, predictions_train[:,1])

0.8149781854383504

In [23]:
roc_auc_score(ytest, predictions[:,1])


0.8067385324287026

### 2. lgbm

In [24]:
import lightgbm as lgbm

train_data = lgbm.Dataset(xtrain, ytrain)
test_data = lgbm.Dataset(xtest, ytest)

# define parameters
parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'feature_fraction': 0.5, 
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'num_threads' : 2,
    'seed' : 76
}

# train lightGBM model
model = lgbm.train(parameters,
                   train_data,
                   valid_sets=test_data,
                   num_boost_round=1000,
                   early_stopping_rounds=20)


[1]	valid_0's auc: 0.825339
Training until validation scores don't improve for 20 rounds
[2]	valid_0's auc: 0.862627
[3]	valid_0's auc: 0.879742
[4]	valid_0's auc: 0.88872
[5]	valid_0's auc: 0.8957
[6]	valid_0's auc: 0.898455
[7]	valid_0's auc: 0.903065
[8]	valid_0's auc: 0.906674
[9]	valid_0's auc: 0.908454
[10]	valid_0's auc: 0.911611
[11]	valid_0's auc: 0.913355
[12]	valid_0's auc: 0.915448
[13]	valid_0's auc: 0.917782
[14]	valid_0's auc: 0.919124
[15]	valid_0's auc: 0.921506
[16]	valid_0's auc: 0.922407
[17]	valid_0's auc: 0.923376
[18]	valid_0's auc: 0.924301
[19]	valid_0's auc: 0.925784
[20]	valid_0's auc: 0.926643
[21]	valid_0's auc: 0.928504
[22]	valid_0's auc: 0.930029
[23]	valid_0's auc: 0.931174
[24]	valid_0's auc: 0.932253
[25]	valid_0's auc: 0.933456
[26]	valid_0's auc: 0.934536
[27]	valid_0's auc: 0.93511
[28]	valid_0's auc: 0.935931
[29]	valid_0's auc: 0.936777
[30]	valid_0's auc: 0.93726
[31]	valid_0's auc: 0.938171
[32]	valid_0's auc: 0.939204
[33]	valid_0's auc: 0.939

In [44]:
pred_train = model.predict(xtrain)

In [45]:
roc_auc_score(ytrain, pred_train)

0.9998562201423977

We can observe that node2vec with Light GBM has higher AUC score, thus works better at predicting the link than logistic regression. 

**References:**
    
[1] Linyuan Lu ̈, Tao Zhou. Link prediction in complex networks: A survey. Physica A: statistical mechanics and its applications 390, 6, 1150–1170, 2011.

[2] David Liben-Nowell and Jon Kleinberg. The link-prediction problem for social networks. Journal of the Amer- ican society for information science and technology 58, 7, 1019–1031, 2007.

[3] M. Fire, L. Tenenboim, O. Lesser, R. Puzis, L. Rokach and Y. Elovici. Link Prediction in Social Networks Using Computationally Efficient Topological Features. 2011 IEEE Third International Conference on Privacy, Security, Risk and Trust and 2011 IEEE Third International Conference on Social Computing, Boston, MA, 2011.

[4] Muhan Zhang and Yixin Chen. Link prediction based on graph neural networks. In Advances in Neural Information Processing Systems, 5165–5175, 2018.

[5] Aditya Grover and Jure Leskovec. node2vec: Scalable Feature Learning for Networks. Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, 2016.

[6] Guolin Ke, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, Tie-Yan Liu. LightGBM: A Highly Efficient Gradient Boosting Decision Tree. Advances in Neural Information Processing Systems 30, 3149-3157, 2017.

[7] Muhan Zhang, Zhicheng Cui, Marion Neumann, Yixin Chen. An End-to-End Deep Learning Architecture for Graph Classification. AAAI Conference on Artificial Intelligence, 2018.

[8] Chami, Ines, Rex Ying, Christopher R ́e and Jure Leskovec. Hyperbolic Graph Convolutional Neural Networks. Advances in neural information processing systems 32, 4869-4880, 2019.

[9] Zonghan Wu, Shirui Pan, Fengwen Chen, Guodong Long, Chengqi Zhang and Philip S. Yu. A Comprehensive Survey on Graph Neural Networks. IEEE Transactions on Neural Networks and Learning Systems, 2020.